### Задание: 
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB 
2. Записать собранные с сайта данные в MongoDB и в какую-нибудь реляционную БД. Сравнить удобство использования БД.
3. Реализовать функцию, записывающую собранные с сайта данные в MongoDB.
4. Написать  функцию, которая будет добавлять в вашу базу данных только новые данные с сайта. (Пока не реализована, думаю как реализовать.)

Все что закоментировано, можете не читать, там просто разные функции к улучшению чтения в БД

### Взят блок с XPath для парсинга данных с сайта

In [29]:
from lxml import html
import requests
import pandas as pd
from datetime import datetime, timedelta
import hashlib
from urllib.parse import urljoin

# Получение страницы
url = "https://stopgame.ru/blogs/all1"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}
response = requests.get(url, headers=headers)
tree = html.fromstring(response.content)

# Парсинг блогов с XPath
blog_cards = tree.xpath('//article[contains(@class, "_card_1lcny_4")]')

In [30]:
games_blogs_data = []
for card in blog_cards:
    try:
        # Название блога
        title_elem = card.xpath('.//a[contains(@class, "_title_1lcny_24")]')
        title = title_elem[0].text_content().strip() if title_elem else "No title"
        
        # Ссылка на блог
        link_elem = card.xpath('.//a[contains(@class, "_title_1lcny_24")]/@href')
        link = "https://stopgame.ru" + link_elem[0] if link_elem else "No link"
        
        # Автор
        author_elem = card.xpath('.//span[@class="_user-info__name_g4zbt_1176"]')
        author = author_elem[0].text_content().strip() if author_elem else "Unknown"
        
        # Ссылка на страницу автора
        author_link_elem = card.xpath('.//a[contains(@class, "_user-info_g4zbt_1124")]/@href')
        author_link = "https://stopgame.ru" + author_link_elem[0] if author_link_elem else "No author link"
        
        # Дата публикации
        date_elem = card.xpath('.//section[contains(@class, "_date_1lcny_225")]')
        date = date_elem[0].text_content().strip() if date_elem else "No date"
        
        # Рейтинг
        rating_elem = card.xpath('.//div[contains(@class, "_rating_1lcny_92")]')
        rating = rating_elem[0].text_content().strip() if rating_elem else "0"
        
        # Количество комментариев
        comments_elem = card.xpath('.//a[contains(@class, "_info__attribute_1lcny_241")][last()]')
        comments = comments_elem[0].text_content().strip().split()[0] if comments_elem else "0"
        
        # Тип контента (Блог, Обзоры и рассуждения)
        content_type_elem = card.xpath('.//section[contains(@class, "_section_1lcny_122")]/span')
        content_type = content_type_elem[0].text_content().strip() if content_type_elem else "Блог"
        
        # Генерация уникального data_key на основе ссылки
        data_key = hashlib.md5(link.encode()).hexdigest() if link != "No link" else hashlib.md5(f"{title}_{author}_{date}".encode()).hexdigest()
        
        games_blogs_data.append({
            'title': title,
            'author': author,
            'date': date,
            'rating': rating,
            'comments': comments,
            'content_type': content_type,
            'link': link,
            'data_key': data_key 
        })
    except Exception as e:
        print(f"Error parsing card: {e}")
        continue


In [31]:
# Создание DataFrame
df = pd.DataFrame(games_blogs_data)
print(f"Найдено блогов: {len(df)}")
print("\nПервые 5 записей:")
print(df.head())

Найдено блогов: 18

Первые 5 записей:
                                               title                author  \
0  Как я 34 игры за 2 года прошла (Часть 6: Resid...  Александра Штейнберг   
1                                            Misery…                    IC   
2              Мультизадачность как игровая механика                Sapies   
3            Steam Deck — всё ещё лучшая портативка?          Улетный гусь   
4                 Такой противоречивый Silent Hill ƒ   Кардинал Кардиналыч   

      date rating comments          content_type  \
0  Сегодня     +4        4  Обзоры и рассуждения   
1  Сегодня     +1        0                  Блог   
2  Сегодня     +5        4               Gamedev   
3    Вчера     +2       29                  Блог   
4    Вчера    +15       12                  Блог   

                                                link  \
0  https://stopgame.ru/blogs/topic/119437/kak_ya_...   
1      https://stopgame.ru/blogs/topic/119431/misery   
2  https://s

### Сохранение даных в нереляционную БД - MongoDB

In [32]:
import pymongo
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
import re

In [33]:
# Функция для подключения к MongoDB
def connect_to_mongodb(connection_string="mongodb://localhost:27017/", db_name="game_blog"):
    """
    Подключение к MongoDB
    """
    try:
        client = MongoClient(connection_string)
        db = client[db_name]
        print(f"Успешное подключение к MongoDB: {db_name}")
        return client, db
    except Exception as e:
        print(f"Ошибка подключения к MongoDB: {e}")
        return None, None

In [34]:
# Функция для преобразования рейтинга из строки в число, нужно для слудющего задания
def parse_rating(rating_str):
    if isinstance(rating_str, (int, float)):
        return rating_str
    
    if not isinstance(rating_str, str):
        return 0
    
    try:
        # Убираем все нецифровые символы кроме минуса
        cleaned = re.sub(r'[^\d+-]', '', rating_str)
        
        if cleaned.startswith('+'):
            return int(cleaned[1:])
        elif cleaned.startswith('-'):
            return -int(cleaned[1:])
        else:
            return int(cleaned)
    except (ValueError, TypeError):
        return 0

In [35]:
# Функция для преобразования кол-во комментариев из строки в число, нужно для слудющего задания
def count_comments(comments_data):
    if comments_data is None:
        return 0
    
    # Если уже число - возвращаем как есть
    if isinstance(comments_data, (int, float)):
        return int(comments_data)
    
    # Если строка - пытаемся преобразовать в число
    if isinstance(comments_data, str):
        try:
            # Убираем все нецифровые символы
            cleaned = re.sub(r'[^\d]', '', comments_data)
            return int(cleaned) if cleaned else 0
        except (ValueError, TypeError):
            return 0
    
    # Если список или другая коллекция - возвращаем длину
    if hasattr(comments_data, '__len__'):
        return len(comments_data)
    
    return 0

In [36]:
# Функция для преобразования данных в формат MongoDB
def prepare_blog_data(blog_data):
    """
    Подготовка данных блога для MongoDB
    """
    # Преобразуем рейтинг в число
    rating = parse_rating(blog_data.get("rating", 0))

    comments = count_comments(blog_data.get("comments", 0))
    
    return {
        "title": blog_data.get("title", ""),
        "author": blog_data.get("author", ""),
        "rating": rating, 
        "comments": comments, 
        "content_type": blog_data.get("content_type", ""),
        "date": blog_data.get("date", ""),
        "scraped_at": datetime.now(),
        "data_key": blog_data.get("data_key", "")
    }

In [38]:
# Функция для проверки существующих данных и добавления только новых
def save_only_new_to_mongodb(db, collection_name, blog_data_list):
    """
    Добавляет в базу данных только новые записи (на основе data_key)
    """
    try:
        collection = db[collection_name]
        new_records = []
        existing_count = 0
        
        for blog_data in blog_data_list:
            data_key = blog_data.get('data_key')
            
            # Проверяем, существует ли запись с таким data_key
            existing_record = collection.find_one({"data_key": data_key})
            
            if not existing_record:
                # Если записи нет - добавляем в список новых
                prepared_data = prepare_blog_data(blog_data)
                new_records.append(prepared_data)
            else:
                existing_count += 1
        
        # Вставляем только новые записи
        if new_records:
            result = collection.insert_many(new_records)
            print(f"Добавлено {len(result.inserted_ids)} новых записей")
            print(f"Пропущено {existing_count} существующих записей")
            return result.inserted_ids
        else:
            print(f"Все записи уже существуют в базе. Новых записей: 0")
            return []
            
    except Exception as e:
        print(f"Ошибка при добавлении новых записей в MongoDB: {e}")
        return None

In [ ]:
# Функция для записи данных в MongoDB
def save_to_mongodb(db, collection_name, blog_data_list):
    """
    Сохранение данных блогов в MongoDB
    """
    try:
        collection = db[collection_name]
        
        # Подготавливаем данные
        prepared_data = [prepare_blog_data(blog) for blog in blog_data_list]
        
        # Вставляем данные
        result = collection.insert_many(prepared_data)
        print(f"Успешно сохранено {len(result.inserted_ids)} записей в MongoDB")
        return result.inserted_ids
        
    except Exception as e:
        print(f"Ошибка при сохранении в MongoDB: {e}")
        return None

In [39]:
# Функция для чтения данных из MongoDB
def read_from_mongodb(db, collection_name, query={}, limit=0):
    """
    Чтение данных из MongoDB
    """
    try:
        collection = db[collection_name]
        
        if limit > 0:
            cursor = collection.find(query).limit(limit)
        else:
            cursor = collection.find(query)
            
        data = list(cursor)
        print(f"Прочитано {len(data)} записей из MongoDB")
        return data
        
    except Exception as e:
        print(f"Ошибка при чтении из MongoDB: {e}")
        return []


In [40]:
# Функция для преобразования данных MongoDB в DataFrame
def mongodb_to_dataframe(db, collection_name, query={}, limit=0):
    """
    Преобразование данных из MongoDB в pandas DataFrame
    """
    try:
        data = read_from_mongodb(db, collection_name, query, limit)
        
        # Убираем поле _id для DataFrame
        for item in data:
            item.pop('_id', None)
            item.pop('scraped_at', None)  # Убираем служебное поле
        
        df = pd.DataFrame(data)
        return df
        
    except Exception as e:
        print(f"Ошибка при преобразовании в DataFrame: {e}")
        return pd.DataFrame()

In [41]:
# Функция для очистки коллекции (опционально)
def clear_collection(db, collection_name):
    """
    Очистка коллекции MongoDB
    """
    try:
        collection = db[collection_name]
        result = collection.delete_many({})
        print(f"Удалено {result.deleted_count} записей из коллекции {collection_name}")
        return result.deleted_count
    except Exception as e:
        print(f"Ошибка при очистке коллекции: {e}")
        return 0

In [42]:
client, db = connect_to_mongodb()
    
if db is not None:
        collection_name = "game_blogs"
        
        print("\\n=== ТЕСТИРОВАНИЕ РАБОТЫ С MONGODB ===\\n")
        
        # 1. Сохраняем все данные (первоначальная загрузка)
        print("1. Первоначальное сохранение данных:")
        save_to_mongodb(db, collection_name, games_blogs_data)
        
        # 2. Пытаемся сохранить те же данные еще раз - должны добавиться только новые
        print("\\n2. Попытка повторного сохранения (должны добавиться только новые):")
        save_only_new_to_mongodb(db, collection_name, games_blogs_data)
        
        # 3. Читаем данные обратно
        print("\\n3. Чтение данных из MongoDB:")
        all_data = read_from_mongodb(db, collection_name)
        
        # 4. Преобразуем в DataFrame для удобства просмотра
        print("\\n4. Данные в формате DataFrame:")
        df_from_mongo = mongodb_to_dataframe(db, collection_name, limit=5)
        if not df_from_mongo.empty:
            print(df_from_mongo.head())
        
        # 5. Пример фильтрации данных
        print("\\n5. Блоги с рейтингом больше 0:")
        high_rated = mongodb_to_dataframe(db, collection_name, 
                                        query={"rating": {"$gt": 0}}, 
                                        limit=5)
        if not high_rated.empty:
            print(high_rated[['title', 'author', 'rating']].head())
        
        # 6. Статистика по коллекции
        print("\\n6. Статистика коллекции:")
        collection = db[collection_name]
        total_count = collection.count_documents({})
        blog_count = collection.count_documents({"content_type": "Блог"})
        review_count = collection.count_documents({"content_type": "Обзоры и рассуждения"})
        
        print(f"Всего записей: {total_count}")
        print(f"Блогов: {blog_count}")
        print(f"Обзоров: {review_count}")
        
        # Закрываем соединение
        client.close()
        print("\\nСоединение с MongoDB закрыто")

Успешное подключение к MongoDB: game_blog
\n=== ТЕСТИРОВАНИЕ РАБОТЫ С MONGODB ===\n
1. Первоначальное сохранение данных:
Успешно сохранено 18 записей в MongoDB
\n2. Попытка повторного сохранения (должны добавиться только новые):
Все записи уже существуют в базе. Новых записей: 0
\n3. Чтение данных из MongoDB:
Прочитано 54 записей из MongoDB
\n4. Данные в формате DataFrame:
Прочитано 5 записей из MongoDB
                                               title                author  \
0  Как я 34 игры за 2 года прошла (Часть 6: Resid...  Александра Штейнберг   
1                                            Misery…                    IC   
2              Мультизадачность как игровая механика                Sapies   
3            Steam Deck — всё ещё лучшая портативка?          Улетный гусь   
4                 Такой противоречивый Silent Hill ƒ   Кардинал Кардиналыч   

   rating  comments          content_type     date  \
0       2         4  Обзоры и рассуждения  Сегодня   
1       1       